In [14]:
import sqlite3 as lite
import pandas as pd

con = lite.connect('./data/checking-logs.sqlite') # подключение БД к Питон

pd.io.sql.read_sql("PRAGMA table_info(test)", con) # PRAGMA получение информации о полях таблицы

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [15]:
pd.io.sql.read_sql("""
    SELECT * FROM test 
    LIMIT 10
    """,con) # первые 10 строк таблицы

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [16]:
df_min = pd.io.sql.read_sql("""
    SELECT t.uid, cast(min(julianday(t.first_commit_ts) - julianday(d.deadlines, 'unixepoch')) * 24 AS integer) AS minimum
    FROM test AS t LEFT JOIN deadlines AS d
    ON t.labname = d.labs
    WHERE t.labname NOT IN ('project1') 
    """, con) #julianday возвращает дату в виде числа с плавающей запятой
# cast преобразует выражение из одного типа данных в другой тип данных
# unixepoch определяет момент времени, в который значение времени в формате Unix равно 0
df_min.to_sql('df_min', con, if_exists = 'replace')
df_min

,uid,minimum
0,user_30,-202


In [17]:
df_max = pd.io.sql.read_sql("""
    SELECT t.uid, cast(max(julianday(t.first_commit_ts) - julianday(d.deadlines, 'unixepoch')) * 24 AS integer) AS maximum 
    FROM test AS t LEFT JOIN deadlines AS d
    ON t.labname = d.labs
    WHERE t.labname NOT IN ('project1') 
    """, con)
df_max.to_sql('df_max', con, if_exists = 'replace')
df_max

,uid,maximum
0,user_25,-2


In [18]:
df_avg = pd.io.sql.read_sql("""
    SELECT avg(cast((julianday(t.first_commit_ts) - julianday(d.deadlines, 'unixepoch')) * 24 AS integer)) AS average 
    FROM test AS t LEFT JOIN deadlines AS d
    ON t.labname = d.labs
    WHERE t.labname NOT IN ('project1') 
    """, con)
df_avg.to_sql('df_avg', con, if_exists = 'replace')
df_avg

,average
0,-89.125


In [19]:
views_diff = pd.io.sql.read_sql("""
    SELECT DISTINCT t. uid, diff_avg, pageviews
    FROM (SELECT uid, avg(cast((julianday(t.first_commit_ts) - julianday(d.deadlines, 'unixepoch')) * 24 AS integer)) AS diff_avg
        FROM test AS t LEFT JOIN deadlines AS d
        ON t.labname = d.labs
        WHERE t.labname NOT IN ('project1')
        GROUP BY uid) AS t
            LEFT JOIN 
            (SELECT uid, count(datetime) AS pageviews
            FROM pageviews
            GROUP by uid) AS p
            ON t.uid = p.uid
    """, con)
views_diff.to_sql('views_diff', con, if_exists = 'replace')
views_diff[['diff_avg', 'pageviews']].corr()

,diff_avg,pageviews
diff_avg,1.000000,-0.279736
pageviews,-0.279736,1.000000


In [20]:
con.close()